1.
Посчитайте, сколько компаний закрылось.

SELECT count(status) FROM company WHERE status = 'closed'; 

2.

Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total .

SELECT funding_total

FROM company 

Where category_code = 'news'

AND country_code = 'USA'

order by funding_total DESC

3.
Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

SELECT sum(price_amount )

FROM acquisition

WHERE 

term_code = 'cash'

AND (EXTRACT(YEAR FROM CAST(acquired_at  AS timestamp)) = '2011'

OR EXTRACT(YEAR FROM CAST(acquired_at  AS timestamp)) = '2012'

OR EXTRACT(YEAR FROM CAST(acquired_at  AS timestamp)) = '2013');

4.
Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.

SELECT first_name,

last_name,

twitter_username

FROM people

WHERE twitter_username LIKE 'Silver%'


5.
Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку 'money', а фамилия начинается на 'K'.

SELECT *

FROM people

WHERE twitter_username LIKE '%money%'

AND last_name LIKE 'K%'

6.
Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

SELECT SUM(funding_total), country_code 
FROM company

GROUP BY country_code

ORDER BY SUM(funding_total) DESC

7.
Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

SELECT funded_at,

MIN(raised_amount),

MAX(raised_amount)

FROM funding_round

GROUP BY funded_at
HAVING MIN(raised_amount) <> 0

AND MIN(raised_amount) <> MAX(raised_amount);


8.
Создайте поле с категориями:
Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.
Отобразите все поля таблицы fund и новое поле с категориями.

SELECT *,

CASE

    WHEN invested_companies > 100  THEN 'high_activity'
    
    WHEN invested_companies >= 20 AND invested_companies < 100  THEN 'middle_activity' 
    
    WHEN invested_companies < 20  THEN 'low_activity' 
END

FROM fund;

9.
Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

SELECT 
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           
           WHEN invested_companies >=20 THEN 'middle_activity'
           
           ELSE 'low_activity'
       
       END AS activity , ROUND(AVG(investment_rounds))

FROM fund

GROUP BY activity

ORDER BY AVG(investment_rounds) ASC;

10.
Выгрузите таблицу с десятью самыми активными инвестирующими странами. Активность страны определите по среднему количеству компаний, в которые инвестируют фонды этой страны.
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды, основанные с 2010 по 2012 год включительно.
Исключите из таблицы страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. Отсортируйте таблицу по среднему количеству компаний от большего к меньшему, а затем по коду страны в лексикографическом порядке.
Для фильтрации диапазона по годам используйте оператор BETWEEN.

SELECT country_code, MIN(invested_companies ), MAX(invested_companies ), AVG(invested_companies )
       
FROM fund

WHERE EXTRACT(YEAR FROM founded_at  ) BETWEEN 2010 AND 2012

GROUP BY country_code

HAVING MIN(invested_companies) <> 0

ORDER BY AVG(invested_companies) DESC

LIMIT 10

11.
Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

SELECT p.first_name , p.last_name, ed.instituition

FROM  people AS P

left JOIN  education AS ed ON p.id = ed.person_id 

12.
Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

SELECT company.name , COUNT(DISTINCT ed.instituition) as con

FROM company

inner JOIN  people AS p ON company.id  = p.company_id  

inner JOIN  education AS ed ON p.id = ed.person_id  

GROUP BY company.name

ORDER BY con DESC 

LIMIT 5

13.

Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

WITH c AS (SELECT   name
           
           FROM company
 
 left JOIN funding_round AS fr ON company.id  = fr.company_id
              
              WHERE status = 'closed'
        
        AND fr.is_last_round = 1
          
        AND fr.is_first_round=1)

SELECT DISTINCT(c.name)

FROM c

GROUP BY c.name


14.

Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

WITH c AS (WITH fr AS (SELECT company_id, funded_at, is_first_round, is_last_round

           FROM funding_round
           
           WHERE is_last_round = 1
           
          AND is_first_round=1)
          
          SELECT DISTINCT name
          
          FROM company
          
 INNER JOIN fr  ON company.id  = fr.company_id
 
            WHERE status = 'closed')
          
SELECT DISTINCT people.id
FROM people

INNER JOIN company  ON people.company_id  = company.id

INNER JOIN c  ON company.name  = c.name

GROUP BY people.id;

15.

Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

WITH c AS (WITH fr AS (SELECT company_id, funded_at, is_first_round, is_last_round

           FROM funding_round
           
           WHERE is_last_round = 1
           
          AND is_first_round=1)
          
          SELECT DISTINCT name
          
          FROM company
          
 INNER JOIN fr  ON company.id  = fr.company_id
 
            WHERE status = 'closed')
          
SELECT DISTINCT people.id, ed.instituition

FROM people

INNER JOIN company  ON people.company_id  = company.id

INNER JOIN c  ON company.name  = c.name

INNER JOIN  education AS ed ON people.id  = ed.person_id 

GROUP BY people.id,ed.instituition

16.
Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

WITH c AS (WITH fr AS (SELECT company_id, funded_at, is_first_round, is_last_round

           FROM funding_round
          
           WHERE is_last_round = 1
           
          AND is_first_round=1)
          
          SELECT DISTINCT name
          
          FROM company
          
 INNER JOIN fr  ON company.id  = fr.company_id
 
            WHERE status = 'closed')
          
SELECT DISTINCT people.id,  COUNT(ed.instituition)

FROM people

INNER JOIN company  ON people.company_id  = company.id

INNER JOIN c  ON company.name  = c.name

INNER JOIN  education AS ed ON people.id  = ed.person_id 

GROUP BY people.id

17.
Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

WITH c AS (WITH fr AS (SELECT company_id, funded_at, is_first_round, is_last_round

           FROM funding_round
           
           WHERE is_last_round = 1
           
          AND is_first_round=1)
          
          SELECT DISTINCT name
          
          FROM company
          
 INNER JOIN fr  ON company.id  = fr.company_id
 
            WHERE status = 'closed')
            
    SELECT AVG(k.sr)
    
    from 
    
(SELECT DISTINCT people.id,  COUNT(ed.instituition) AS sr

FROM people


INNER JOIN company  ON people.company_id  = company.id

INNER JOIN c  ON company.name  = c.name

INNER JOIN  education AS ed ON people.id  = ed.person_id 

GROUP BY people.id) AS K


18.
Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook*.
*(сервис, запрещённый на территории РФ)

WITH tot AS (SELECT COUNT(e.instituition) AS total_instituition
             
             FROM people AS p JOIN education AS e ON p.id = e.person_id 
             
             WHERE company_id in (SELECT id 
             
	                              FROM company 
                                  
	                              WHERE name LIKE 'Facebook' 
                                  
                                  GROUP BY id)
             GROUP BY e.person_id)
             
SELECT AVG(tot.total_instituition)

FROM tot
	;

19.

Составьте таблицу из полей:
name_of_fund — название фонда;
name_of_company — название компании;
amount — сумма инвестиций, которую привлекла компания в раунде.
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

select f.name as name_of_fund, 

c.name as name_of_company,

fr.raised_amount as amount


from investment as i

inner join company as c on i.company_id = c.id

inner join fund as f on  f.id = i.fund_id 

inner join funding_round as fr on i.funding_round_id = fr.id


where extract (year from cast(fr.funded_at as date)) 

BETWEEN '2012' AND '2013' and  c.milestones> 6 

20.
Выгрузите таблицу, в которой будут такие поля:
название компании-покупателя;
сумма сделки;
название компании, которую купили;
сумма инвестиций, вложенных в купленную компанию;
доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.
Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы.
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

SELECT autre.name AS acquiring_company, ashet.name AS acquired_company, a.price_amount , ashet.funding_total, ROUND(a.price_amount /ashet.funding_total) AS propothin

FROM acquisition AS a 

LEFT JOIN company AS ashet ON a.acquired_company_id   = ashet.id

LEFT JOIN company AS autre ON a.acquiring_company_id  = autre.id

WHERE a.price_amount <> 0

AND ashet.funding_total <> 0

ORDER BY a.price_amount  DESC, acquired_company

LIMIT 10


21.
Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

select com.name, extract(month from fun.funded_at) as month

from company as com

full join funding_round as fun on com.id = fun.company_id

where com.category_code  = 'social' and extract(year from cast(fun.funded_at  as date)) in (2010,2011,2012,2013)

22.
Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
номер месяца, в котором проходили раунды;
количество уникальных названий фондов из США, которые инвестировали в этом месяце;
количество компаний, купленных за этот месяц;
общая сумма сделок по покупкам в этом месяце.

SELECT 

fs.count_month,

funds,

company_count,

amount

FROM 

(SELECT

COUNT(acquired_company_id ) as company_count,

SUM(price_amount) AS  amount,

EXTRACT(month from acquired_at ) as count_month

from acquisition

WHERE  EXTRACT(YEAR FROM acquired_at) BETWEEN 2010 AND 2013

GROUP BY extract(month from acquired_at))  as a

inner join (select 

extract(month from fr.funded_at ) as count_month, 

count(distinct f.name) as funds       

from company as c

INNER JOIN investment as i ON i.company_id = c.id

INNER JOIN funding_round as fr ON fr.id = i.funding_round_id

INNER JOIN fund as f ON f.id = i.fund_id

WHERE f.country_code = 'USA'

AND EXTRACT(YEAR FROM funded_at) BETWEEN 2010 AND 2013

GROUP BY extract(month from funded_at)) as fs  ON a.count_month = fs.count_month

23.
Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

SELECT year_2011.country_code,

        year_2011.total_2011,
        
        year_2012.total_2012,
        
        year_2013.total_2013
        
FROM

    (SELECT
    
             country_code,
             
            AVG(funding_total) AS total_2011
            
    FROM company
    
    WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) ='2011'
    
    GROUP BY country_code) AS year_2011
    
 JOIN
 
    (SELECT
    
             country_code,
             
            AVG(funding_total) AS total_2012
            
     FROM company
     
     WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) ='2012'
     
     GROUP BY country_code) AS year_2012
     
     ON year_2011.country_code = year_2012.country_code
     
JOIN

    (SELECT
    
             country_code,
             
            AVG(funding_total) AS total_2013
            
     FROM company
     
     WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) ='2013'
     
     GROUP BY country_code) AS year_2013
     
     ON year_2011.country_code = year_2013.country_code
     
     ORDER BY year_2011.total_2011 DESC;